In [27]:
test = """2,2,2
1,2,2
3,2,2
2,1,2
2,3,2
2,2,1
2,2,3
2,2,4
2,2,6
1,2,5
3,2,5
2,1,5
2,3,5"""
source = test

In [28]:
test="""2,2,1
1,2,2
2,1,2
3,2,2
2,3,2
2,2,3"""
# source = test

In [29]:
source = open("./day_18.txt").read()

## Part 1: Surface area

In [30]:
sides = {}
for dims in [list(map(int, line.split(","))) for line in source.splitlines()]:
    strs = [str(d) for d in dims]
    d = ",".join(strs)
    key = "x" + d
    sides[key] =   sides.get(key,0)+1
    key = "y" + d
    sides[key] =   sides.get(key,0)+1
    key = "z" + d
    sides[key] =   sides.get(key,0)+1
    strx = [str(dims[0]+1), strs[1], strs[2]]
    key = "x" + ",".join(strx)
    sides[key] =   sides.get(key,0)-1
    stry = [strs[0], str(dims[1]+1), strs[2]]
    key = "y" + ",".join(stry)
    sides[key] =   sides.get(key,0)-1
    strz = [strs[0], strs[1], str(dims[2]+1)]
    key = "z" + ",".join(strz)
    sides[key] =   sides.get(key,0)-1

goodSides =([(s, sides[s]) for s in sides.keys() if abs(sides[s]) == 1])
print(len(goodSides))
goodSides.sort()
# goodSides
# len(sides), sides

4512


## Part 2: Outside only

In [31]:
minmax = [[min(d), max(d)] for d in zip(*[list(map(int, line.split(","))) for line in source.splitlines()])]

In [32]:
def adjacentSides(sideKey, direction, sides, isDebug=False):
    if isDebug: print("Starting with", sideKey, direction)
    dim = sideKey[0]
    x,y,z = list(map(int, sideKey[1:].split(",")))
    # print(dim, x,y,z)
    asides = []
    if(dim == "x"):
        xd = x -1 if direction == 1 else x
        walls = [(f"y{xd},{y},{z}",-1),    (f"y{xd},{y+1},{z}",1),   (f"z{xd},{y},{z}",-1),  (f"z{xd},{y},{z+1}", 1)]
        tempflats = [(f"x{x},{y-1},{z}",direction),(f"x{x},{y+1},{z}",direction), (f"x{x},{y},{z-1}", direction),(f"x{x},{y},{z+1}", direction)]
        xd += direction
        nextset =  [(f"y{xd},{y},{z}",1),  (f"y{xd},{y+1},{z}",-1), (f"z{xd},{y},{z}", 1),  (f"z{xd},{y},{z+1}", -1)]
               
    if(dim == "y"):
        yd = y -1 if direction == 1 else y
        walls = [(f"x{x},{yd},{z}",-1),    (f"x{x+1},{yd},{z}",1),   (f"z{x},{yd},{z}",-1),  (f"z{x},{yd},{z+1}", 1)]
        tempflats = [(f"y{x-1},{y},{z}",direction),(f"y{x+1},{y},{z}",direction), (f"y{x},{y},{z-1}", direction),(f"y{x},{y},{z+1}", direction)]
        yd += direction
        nextset = [(f"x{x},{yd},{z}",1),    (f"x{x},{yd},{z}",-1),   (f"z{x},{yd},{z}",1),  (f"z{x},{yd},{z+1}", -1)]
    if(dim == "z"):
        zd = z -1 if direction == 1 else z
        walls = [(f"x{x},{y},{zd}",-1),    (f"x{x+1},{y},{zd}",1),   (f"y{x},{y},{zd}",-1),  (f"y{x},{y+1},{zd}", 1)]
        tempflats = [(f"z{x},{y-1},{z}",direction),(f"z{x},{y+1},{z}",direction), (f"z{x-1},{y},{z}",direction),(f"z{x+1},{y},{z}", direction)]
        zd += direction
        nextset = [(f"x{x},{y},{zd}",1),    (f"x{x+1},{y},{zd}",-1),   (f"y{x},{y},{zd}",1),  (f"y{x},{y+1},{zd}", -1)]
        
    if(isDebug):
        print("walls", walls)
        print("tempflats", tempflats)
        print("nextset", nextset)
    asides = [s if sides.get(s[0]) == s[1] else None for s in walls]
    if(isDebug):    print("asides" , asides)
    asides = [asides[i] if asides[i] != None else tempflats[i] if sides.get(tempflats[i][0]) == tempflats[i][1] else None for i in range(len(tempflats))]
    if(isDebug):print("asides after tempflats"  , asides)
    asides = [asides[i] if asides[i] != None else nextset[i] if sides.get(nextset[i][0]) == nextset[i][1] else None for i in range(len(nextset))]
    if(isDebug):print("asides after nextset"  , asides)
    return asides

In [33]:

#for thing in 
scannedPoints = {}
paintedSides = {}
# z = minmax[2][0]
# print(minmax)
for x,y,z in [(x,y,z) for y in range(minmax[1][0], minmax[1][1]+1) for x in range(minmax[0][0], minmax[0][1]+1) for z in range(minmax[2][0], minmax[2][1]+1)]:
    # print(x,y, z)
    if(scannedPoints.get(",".join([str(x),str(y),str(z)])) != None):
        continue
    # print(x,y)
    scannedPoints[",".join([str(x),str(y),str(z)])] = 1 if f"z{x},{y},{z}" in sides else 0;
    key = f"z{x},{y},{z}"
    # print(key)
    # if(f"x{x},{y},{z}" in sides): paintedSides[f"x{x},{y},{z}"] = 1
    # if(f"y{x},{y},{z}" in sides): paintedSides[f"y{x},{y},{z}"] = 1
    if(f"z{x},{y},{z}" in sides): 
        paintedSides[f"z{x},{y},{z}"] = sides[f"z{x},{y},{z}"]
        break
    # if(f"x{x+1},{y},{z}" in sides): paintedSides[f"x{x+1},{y},{z}"] = 1
    # if(f"y{x},{y+1},{z}" in sides): paintedSides[f"y{x},{y+1},{z}"] = 1
    # if(f"z{x},{y},{z+1}" in sides): paintedSides[f"x{x},{y},{z+1}"] = 1
# scannedPoints
# paintedSides

In [34]:
newCount = 0
killer = 20000
sidesToCheck= paintedSides.keys()
while(True):#len(sidesToCheck) > 0):
    # newCount = len(paintedSides.keys())
    keys = list(sidesToCheck).copy()
    sidesToCheck = []
    # print("Sides to check",len(keys), keys)
    # print("------------------")
    for key in keys:
        # print(key)
        asides = adjacentSides(key, paintedSides[key], sides,False);
        # print(key, asides)
        for obj in asides:
            if obj == None: continue
            side, direction = obj
            # print(side)
            if paintedSides.get(side) == None:
                # print(side)
                if sides[side] == direction: 
                    # if(side in badsides):
                    #     print("BAD MAN BAD BAD BAD")
                    #     killer = -1
                    #     break
                    sidesToCheck.append(side) 
                    paintedSides[side] = direction
                    # print("Added side", side, direction, killer)
                # else:
                    # print("BAD MAN NO LIKE", side)
        if(killer == -1): break
    # newCount = len(paintedSides)
    killer = killer -1
    # print(len(paintedSides), killer)
    # print(paintedSides)
    # if(killer %10)==0: 
    #     print(len(paintedSides), killer)
    if(killer<1):
        print("break")
        break
print("Done")
print(len(paintedSides))
# print(paintedSides)
# l = list(paintedSides.keys())
# l.sort()
# print(l)
# print(paintedSides)

break
Done
2544


## Guesses:
* 2568 (high)
* 2556 (high)
* 2550 (wrong)
* 2545 (wrong)
* 2544 (low)

## New Strategy

In [35]:
import numpy
bounds = [(mm[0]-1, mm[1]+1) for mm in minmax]
# print(bounds)
field = numpy.zeros((bounds[0][1]-bounds[0][0]+1, bounds[1][1]-bounds[1][0]+1, bounds[2][1]-bounds[2][0]+1, 1), dtype=bool)
field2 = numpy.zeros((bounds[0][1]-bounds[0][0]+1, bounds[1][1]-bounds[1][0]+1, bounds[2][1]-bounds[2][0]+1, 1), dtype=int)
# print(len(field))
# print(len(field[0]))
# print(len(field[0][0]))

for xyz in [[int(c) for c in line.split(",")] for line in source.splitlines()]:
    newCoords = numpy.diff(list(zip([numpy.array(bounds)[:,0], numpy.array(xyz)])), axis = 0)[0][0]
    field[newCoords[0], newCoords[1], newCoords[2]] = True
# field





In [36]:
# start at 0,0,0
zzz = numpy.array([0,0,0])
pointsToCheck = [zzz]

def check_point(point):
    if(point[0] < 0 or point[1] < 0 or point[2] < 0): return False
    if(point[0] >= len(field) or point[1] >= len(field[0]) or point[2] >= len(field[0][0])): return False
    return field2[point[0], point[1], point[2]] == 0 and field[point[0], point[1], point[2]] == False 
# print(field2)

while(len(pointsToCheck) > 0):
    # print(pointsToCheck)
    point = pointsToCheck.pop()
    # print(point)
    if(check_point(point)):

        field2[point[0], point[1], point[2]] = 1
        # print("checked", point)
        pointsToCheck.extend([point + numpy.array([0,0,1]), point + numpy.array([0,0,-1]), point + numpy.array([0,1,0]), point + numpy.array([0,-1,0]), point + numpy.array([1,0,0]), point + numpy.array([-1,0,0])])
    # else:
        # print("not checked", point)
    # print("pointsToCheck", pointsToCheck)
# field2


In [37]:
sides = {}
for dims in [(x,y,z) for x in range(len(field2)) for y in range(len(field2[0])) for z in range(len(field2[0][0]))]:
    if(field2[dims[0], dims[1], dims[2]] == 1):
        strs = [str(d) for d in dims]
        d = ",".join(strs)
        key = "x" + d
        sides[key] =   sides.get(key,0)+1
        key = "y" + d
        sides[key] =   sides.get(key,0)+1
        key = "z" + d
        sides[key] =   sides.get(key,0)+1
        strx = [str(dims[0]+1), strs[1], strs[2]]
        key = "x" + ",".join(strx)
        sides[key] =   sides.get(key,0)-1
        stry = [strs[0], str(dims[1]+1), strs[2]]
        key = "y" + ",".join(stry)
        sides[key] =   sides.get(key,0)-1
        strz = [strs[0], strs[1], str(dims[2]+1)]
        key = "z" + ",".join(strz)
        sides[key] =   sides.get(key,0)-1

goodSides =([(s, sides[s]) for s in sides.keys() if abs(sides[s]) == 1])
# print(len(goodSides))
# print(bounds)
exteriorSA = len(field2)* len(field2[0]) * 2 + len(field2)* len(field2[0][0]) * 2 + len(field2[0])* len(field2[0][0]) * 2
# print(exteriorSA)
print(len(goodSides)- exteriorSA)
# goodSides.sort()
# goodSides

# len(sides), sides

2554
